In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def SIFT_Matches(image1, image2, MAX_FEATURES=3000):
    # Convertimos las imágenes de color a escala de grises
    new_image_1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    new_image_2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Creamos un detector SIFT con un límite máximo de características
    sift = cv2.SIFT_create(MAX_FEATURES)

    # Detectamos y calculamos descriptores para las características en ambas imágenes
    Key_Points_1, des1 = sift.detectAndCompute(new_image_1, None)
    Key_Points_2, des2 = sift.detectAndCompute(new_image_2, None)

    # Utilizamos el matcher de fuerza bruta (Brute-Force) para hacer coincidir los descriptores
    Bjf = cv2.BFMatcher()
    matches = Bjf.knnMatch(des1, des2, k=2)

    # Filtramos las coincidencias para mantener solo las que son lo suficientemente buenas
    Correct_Matches = []
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            Correct_Matches.append([m])

    # Extraemos las coordenadas de las características coincidentes en ambas imágenes
    left_coords = np.float32([Key_Points_1[m[0].queryIdx].pt for m in Correct_Matches])
    right_coords = np.float32([Key_Points_2[m[0].trainIdx].pt for m in Correct_Matches])

    return left_coords, right_coords

In [6]:
image1 = cv2.imread("MIT1.jpg")
image2 = cv2.imread("MIT2.jpg")

# Llamar a la función SIFT_Matches
left_coords, right_coords = SIFT_Matches(image1, image2)

# Dibujar puntos en las imágenes
image1_points = image1.copy()
image2_points = image2.copy()

for coord in left_coords:
    x, y = coord
    cv2.circle(image1_points, (int(x), int(y)), 5, (0, 255, 0), -1)  # Dibuja un círculo verde en la imagen1

for coord in right_coords:
    x, y = coord
    cv2.circle(image2_points, (int(x), int(y)), 5, (0, 255, 0), -1) 

# Mostrar las imágenes con los puntos dibujados
cv2.imshow("Image 1 with Points", image1_points)
cv2.imshow("Image 2 with Points", image2_points)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
def FUNDAMENTAL_MATRIX_ESTIMATION(LEFT_IMG_PTS, RIGHT_IMG_PTS):

    A = [] 

    for i in range(8): #Iter over 7 random points
        ul,vl = LEFT_IMG_PTS[i]
        ur,vr = RIGHT_IMG_PTS[i]

        row = np.asarray([ul * ur, ul * vr, ul, vl * ur, vr * vl, vl, ur, vr, 1])

        A.append(row)

    

    A = np.asarray(A)

    U,S,V = np.linalg.svd(A)
    F = np.reshape(V[-1,:],(3,3))

    UF , SF , VF = np.linalg.svd(F)
    SF = np.diag(SF)
    SF[2,2] = 0

    F = UF @ (SF @ VF)

    return F

In [11]:
F = FUNDAMENTAL_MATRIX_ESTIMATION(left_coords,right_coords)
print(F)

[[-7.20230859e-07  9.21322929e-06 -5.81492367e-03]
 [-3.28734304e-07  8.23863406e-07 -3.21515119e-04]
 [ 6.92598728e-05 -1.52495909e-03  9.99981876e-01]]


In [16]:

#CENTERING DE DATA CORRESPONDENCE POINTS
ul_mean = np.mean (left_coords[ : , 0])
vl_mean = np.mean (left_coords[ : , 1])

ur_mean = np.mean (right_coords[ : , 0])
vr_mean = np.mean (right_coords[ : , 1])

In [ ]:
s_l = np.sqrt(2) / ()